In [144]:
import pandas as pd

In [145]:
vote_ratio_df = pd.read_csv("./2nd_preproc_data/vote_ratio_df.csv", index_col=0)
theme_df = pd.read_csv("./2nd_preproc_data/theme_df.csv", index_col=0)

In [146]:
vote_ratio_df.head()

,loc_1,loc_2,arg_1,18_29,30_39,40_49,50_59,60_,time
9,서울특별시,중구,남자,9.3,10.3,8.0,8.8,12.2,2020_04
11,서울특별시,중구,여자,9.1,11.1,8.3,9.2,13.7,2020_04
15,서울특별시,용산구,남자,9.7,9.8,7.4,8.8,12.1,2020_04
17,서울특별시,용산구,여자,10.1,9.4,8.1,9.4,15.1,2020_04
21,서울특별시,성동구,남자,10.2,9.3,8.7,9.6,11.3,2020_04


In [147]:
theme_df.head()

,loc_1,loc_2,loc_3,theme
0,전라남도,곡성군,곡성읍,순천시광양시곡성군구례군을
1,전라남도,곡성군,오곡면,순천시광양시곡성군구례군을
2,전라남도,곡성군,삼기면,순천시광양시곡성군구례군을
3,전라남도,곡성군,석곡면,순천시광양시곡성군구례군을
4,전라남도,곡성군,목사동면,순천시광양시곡성군구례군을


In [148]:
vote_ratio_df = vote_ratio_df.merge(
    theme_df.loc[:, ["loc_1", "loc_2", "theme"]], on=["loc_1", "loc_2"]
)

In [149]:
vote_ratio_df.drop_duplicates(inplace=True)

vote_ratio_gb = vote_ratio_df.groupby(["theme", "time"])[
    ["18_29", "30_39", "40_49", "50_59", "60_"]
].mean()
vote_ratio_gb = vote_ratio_gb.apply(lambda x: x * 2)
vote_ratio_gb.reset_index(inplace=True)
vote_ratio_gb

,theme,time,18_29,30_39,40_49,50_59,60_
0,강남구갑,2020_04,18.7,17.3,19.4,18.7,25.7
1,강남구갑,2022_03,17.7,17.6,19.8,18.6,26.3
2,강남구갑,2022_06,18.2,17.1,20.5,18.8,25.4
3,강남구병,2020_04,18.7,17.3,19.4,18.7,25.7
4,강남구병,2022_03,17.7,17.6,19.8,18.6,26.3
...,...,...,...,...,...,...,...
699,화성시병,2022_03,18.8,20.7,25.1,17.9,17.4
700,화성시병,2022_06,18.2,23.5,25.1,17.0,16.4
701,화성시을,2020_04,19.5,19.7,24.4,18.1,18.3
702,화성시을,2022_03,18.8,20.7,25.1,17.9,17.4


In [150]:
# 나이별 투표 경직도
## [50 ~ 0]
import numpy as np

weights = np.linspace(50, 0, len(vote_ratio_gb.columns))


def weighted_mean(values):
    return np.average(values, weights=range(1, len(values) + 1))

In [151]:
tmp = vote_ratio_gb.drop(columns=["time"])
cols = tmp.columns[1:]

In [152]:
df_wm = lambda col: tmp.groupby("theme")[col].apply(weighted_mean)
weighted_vote_ratio_df = pd.concat(list(map(df_wm, cols)), axis=1)
weighted_vote_ratio_df

,18_29,30_39,40_49,50_59,60_
theme,,,,,
강남구갑,18.116667,17.300000,20.083333,18.716667,25.750000
강남구병,18.116667,17.300000,20.083333,18.716667,25.750000
강남구을,18.116667,17.300000,20.083333,18.716667,25.750000
강동구갑,17.166667,16.950000,19.333333,18.650000,27.733333
강동구을,17.166667,16.950000,19.333333,18.650000,27.733333
...,...,...,...,...,...
홍성군예산군,12.733333,11.208333,16.008333,18.341667,41.666667
홍천군횡성군영월군평창군,10.391667,7.233333,12.320833,20.920833,49.120833
화성시갑,18.616667,21.933333,24.983333,17.483333,17.050000
